# 3. 深層生成モデルの実装例：VAE

In [1]:
from __future__ import print_function
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms

from tqdm import tqdm

batch_size = 128
epochs = 5
seed = 1
torch.manual_seed(seed)

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

## 3.1 Tars.modelsを使わない方法

ここでは，2で説明した確率分布クラスの記述方法を使って， 深層生成モデルの一つであるVAEを実装してみましょう．

VAEでは， 次の生成モデルを考えます．

\begin{equation*}
p_{\theta}(x,z)=p_{\theta}(x|z)p(z)
\end{equation*}

ただし，$x$は観測変数，$z$は潜在変数，$\theta$はパラメータです．

VAEでは，$p_{\theta}(x|z)$は深層ニューラルネットワークで定義されるので，$\theta$はニューラルネットワークのパラメータになります．

VAEの目標は，周辺尤度$p(x)=\int p_\theta(x|z)p(z)dz$を最大化するように生成モデルを学習することです． ただし，これは直接最大化することができません．

そこで，代わりに次のような変分下界$\mathcal{L}(x)$を最大化することで，モデルを学習します．

\begin{equation*}
\log p(x) \geq E_{q_\phi(z|x)}[\log \frac{p_\theta(x,z)}{q_\phi(z|x)}] = \mathcal{L}(x)
\end{equation*}

ただし，$q_\phi(z|x)$は事後分布$p(z|x)$の近似分布で，$\phi$はそのパラメータです．VAEではこの分布も深層ニューラルネットワークでモデル化します（amortized inference）．

それぞれの分布について，$q_\phi(ｚ|x)$は$x$を$z$にエンコードするので**エンコーダ**，$p_\theta(x|z)$は$z$から$x$にデコードするので**デコーダ**と呼ばれます． このことから，このモデルは**variational autoencoders（VAE）**と呼ばれます．

ではこれに従って，Tarsの分布クラスで，各分布を設定しましょう．

In [2]:
from Tars.distributions import Normal, Bernoulli

x_dim = 784
z_dim = 64

# inference model q(z|x)
class Inference(Normal):
    def __init__(self):
        super(Inference, self).__init__(cond_var=["x"], var=["z"])

        self.fc1 = nn.Linear(x_dim, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc31 = nn.Linear(512, z_dim)
        self.fc32 = nn.Linear(512, z_dim)

    def forward(self, x):
        h = F.relu(self.fc1(x))
        h = F.relu(self.fc2(h))
        return {"loc": self.fc31(h), "scale": F.softplus(self.fc32(h))}

    
# generative model p(x|z)    
class Generator(Bernoulli):
    def __init__(self):
        super(Generator, self).__init__(cond_var=["z"], var=["x"])

        self.fc1 = nn.Linear(z_dim, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, x_dim)

    def forward(self, z):
        h = F.relu(self.fc1(z))
        h = F.relu(self.fc2(h))
        return {"probs": F.sigmoid(self.fc3(h))}
    
q = Inference()    
_p = Generator()
    
# prior model p(z)
loc = torch.tensor(0.).to(device)
scale = torch.tensor(1.).to(device)
prior = Normal(loc=loc, scale=scale, var=["z"], dim=z_dim)

ここでは，エンコーダにガウス分布，デコーダにベルヌーイ分布を用いています.

次に，生成モデルの同時分布を設定します．

In [3]:
p = _p * prior

print(p.prob_text)
print(p.prob_factorized_text)

p(x,z)
p(x|z)p(z)


今回は，GPU計算するので，各分布をGPUに載せます．

In [4]:
p.to(device)
q.to(device)

Inference(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc31): Linear(in_features=512, out_features=64, bias=True)
  (fc32): Linear(in_features=512, out_features=64, bias=True)
)

次に，目的関数を設定します． VAEの目的関数は，次の式で表される変分下界でした． 

\begin{equation*}
\mathcal{L}(x) = E_{q_\phi(z|x)}[\log \frac{p_\theta(x,z)}{q_\phi(z|x)}]
\end{equation*}

この計算は，前回の確率分布クラスの特性を使うことで，以下のように非常にシンプルに書くことができます．

In [5]:
def elbo(x):
    #1. sample from q(z|x) 
    samples = q.sample(x)
    
    #2. caluculate the lower bound (log p(x,z) - log q(z|x))
    lower_bound = p.log_likelihood(samples) - q.log_likelihood(samples)

    loss = -torch.mean(lower_bound)

    return loss

この関数は，入力$x$を受け取って上記の式に従って変分下界を計算し，それを誤差関数として返します．

なおpytorchでは，誤差を最小化するように学習しますので，ここでは変分下界にマイナスを付けたものを誤差関数としています．

また，パラメータや最適化手法を設定します．

In [6]:
q_params = list(q.parameters())
p_params = list(p.parameters())
params = q_params + p_params
print(type(q.parameters()))

optimizer = optim.Adam(params, lr=1e-3)

<class 'generator'>


学習用のデータを読み込みます．今回はMNISTを使います．

In [7]:
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)

最後に，trainとtestの関数を設定します．

In [8]:
def train(epoch):
    train_loss = 0
    for batch_idx, (data, _) in enumerate(tqdm(train_loader)):
        data = data.to(device)
        optimizer.zero_grad()
        
        # calculate the ELBO
        loss = elbo({"x": data.view(-1, 784)})
        
        # backprop
        loss.backward()

        # update params
        optimizer.step()             
        
        
        train_loss += loss
 
    train_loss = train_loss * train_loader.batch_size / len(train_loader.dataset)
    print('Epoch: {} Train loss: {:.4f}'.format(epoch, train_loss))
    return train_loss

In [9]:
def test(epoch):
    p.eval()
    q.eval()    
    
    test_loss = 0
    for i, (data, _) in enumerate(test_loader):
        data = data.to(device)
        with torch.no_grad():
            loss = elbo({"x": data.view(-1, 784)})        
        test_loss += loss

    test_loss = test_loss * test_loader.batch_size / len(test_loader.dataset)
    print('Test loss: {:.4f}'.format(test_loss))
    return test_loss

そして，学習を行います．

In [10]:
for epoch in range(1, epochs + 1):
    train_loss = train(epoch)
    test_loss = test(epoch)

100%|██████████| 469/469 [00:05<00:00, 86.23it/s]

Epoch: 1 Train loss: 183.3633



  0%|          | 0/469 [00:00<?, ?it/s]

Test loss: 146.0240


100%|██████████| 469/469 [00:05<00:00, 92.36it/s]


...

Epoch: 4 Train loss: 114.9133



  0%|          | 0/469 [00:00<?, ?it/s]

Test loss: 113.0229


100%|██████████| 469/469 [00:05<00:00, 88.70it/s]


Epoch: 5 Train loss: 111.5798
Test loss: 111.1281


テスト誤差がどんどん小さくなっているのが確認できると思います． これは，変分下界が大きくなっていることを示しており，VAEの学習がうまく進んでいることがわかります．

今回は，エンコーダ（ガウス分布）とデコーダ（ベルヌーイ分布）で構成されるシンプルなモデルとしてVAEを考えましたが，Tarsでは，同時分布をモデル化するAPIを採用することで，**同じ実装方法で任意の形の生成モデルを学習することができます**．

また，それぞれの分布クラスは学習中や学習後でも自由にサンプリングできます．したがって，例えば学習後に生成モデル_pからサンプリングすることで，MNIST画像を生成することができます． 

こちらについてはTars/exampleにいくつか例を用意しましたので，実行してみてください．

## 3.2 Tars.modelsを使う方法

3.1では下界を実装しましたが，TarsではTars.models以下にいくつかの学習モデルを用意しています．

変分推論の場合は，任意の生成モデルについて，Tars.models.VIを使うことで簡単に学習できます．

In [11]:
from Tars.models import VI
model = VI(p, q, optimizer=optim.Adam, optimizer_params={"lr":1e-3})

In [12]:
def train(epoch):
    train_loss = 0
    for batch_idx, (data, _) in enumerate(tqdm(train_loader)):
        data = data.to(device)
        loss = model.train({"x": data.view(-1, 784)})
        train_loss += loss
 
    train_loss = train_loss * train_loader.batch_size / len(train_loader.dataset)
    print('Epoch: {} Train loss: {:.4f}'.format(epoch, train_loss))
    return train_loss

In [13]:
def test(epoch):
    test_loss = 0
    for i, (data, _) in enumerate(test_loader):
        data = data.to(device)
        loss = model.test({"x": data.view(-1, 784)})
        test_loss += loss

    test_loss = test_loss * test_loader.batch_size / len(test_loader.dataset)
    print('Test loss: {:.4f}'.format(test_loss))
    return test_loss

In [14]:
for epoch in range(1, epochs + 1):
    train_loss = train(epoch)
    test_loss = test(epoch)

100%|██████████| 469/469 [00:05<00:00, 88.99it/s]

Epoch: 1 Train loss: 109.6197



  0%|          | 0/469 [00:00<?, ?it/s]

Test loss: 109.1395


100%|██████████| 469/469 [00:05<00:00, 92.32it/s]


...

  0%|          | 0/469 [00:00<?, ?it/s]

Test loss: 106.2629


100%|██████████| 469/469 [00:05<00:00, 84.14it/s]

Epoch: 5 Train loss: 104.6524


Test loss: 105.5771


その他の分布やモデルの例については，Tars/exampleをみてください．